In [ ]:
# For Google Colab
# Install folium package.
!pip install folium

In [ ]:
from google.colab import drive
# mount our Google Drive at /gdrive
drive.mount('/gdrive')
!rm -r sample_data/

In [ ]:
# copy all files from "AI_HW2" directory in Google drive to current directory
!cp -r /gdrive/MyDrive/AI_HW2/* .

In [ ]:
# for google colab
# REMEMBER to execute this line once you've modified any .py code!
# Save the .py code you have modified to your Google Drive
!cp ./*.py /gdrive/MyDrive/AI_HW2/

In [ ]:
# Don't change this part.
# For load graph information and show map
import folium
import pickle
def load_path_graph(path):
    with open('graph.pkl', 'rb') as f:
        graph = pickle.load(f)

    node_pairs = list(zip(path[:-1], path[1:]))
    lines = []
    for edge in graph:
        if (edge['u'], edge['v']) in node_pairs or  (edge['v'], edge['u']) in node_pairs:
            lines.append(edge['geometry'])
    return lines

In [ ]:
# import data from csv file
import csv

edges_distance = {}
speed_limit = {}
maximum_speed_limit = float(0)
with open('edges.csv', newline='') as csv_file:
  rows = list(csv.reader(csv_file, delimiter = ','))
  i = 1
  while i < len(rows):
    if int(rows[i][0]) not in edges_distance:
      edges_distance[int(rows[i][0])] = {}
    edges_distance[int(rows[i][0])][int(rows[i][1])] = float(rows[i][2])

    if int(rows[i][0]) not in speed_limit:
      speed_limit[int(rows[i][0])] = {}
    speed_limit[int(rows[i][0])][int(rows[i][1])] = float(float(rows[i][3]) / 3.6)

    maximum_speed_limit = max(maximum_speed_limit, float(float(rows[i][3]) / 3.6))
    i += 1

heuristic_distance = {}
with open('heuristic.csv', newline='') as csv_file:
  rows = list(csv.reader(csv_file, delimiter = ','))
  i = 1
  while i < len(rows):
    if int(rows[i][0]) not in heuristic_distance:
      heuristic_distance[int(rows[i][0])] = {}
    heuristic_distance[int(rows[i][0])][1079387396] = float(rows[i][1])
    heuristic_distance[int(rows[i][0])][1737223506] = float(rows[i][2])
    heuristic_distance[int(rows[i][0])][8513026827] = float(rows[i][3])
    i += 1

In [ ]:
# Part 5
# Change start ID and end ID.
start = 2270143902
end = 1079387396
# start = 426882161
# end = 1737223506
# start = 1718165260
# end = 8513026827

In [ ]:
def bfs(start, end):
  path = []
  dist = float(0)
  num_visited = int(0)

  previous_node = {}
  bfs_queue = []

  bfs_queue.append(start)
  
  while bfs_queue:
    num_visited += 1

    node = bfs_queue.pop(0)

    if (node == end):
      break

    if node not in edges_distance:
      continue

    for destination in edges_distance[node]:
      if destination == start or destination in previous_node:
        continue
      bfs_queue.append(destination)
      previous_node[destination] = node

  path.insert(0,end)
  while path[0] in previous_node:
    dist += edges_distance[previous_node[path[0]]][path[0]]
    path.insert(0, previous_node[path[0]])
  
  return path, dist, num_visited

# Show the result of BFS
bfs_path, bfs_dist, bfs_visited = bfs(start, end)
print(f'The number of nodes in the path found by BFS: {len(bfs_path)}')
print(f'Total distance of path found by BFS: {bfs_dist} m')
print(f'The number of visited nodes in BFS: {bfs_visited}\n')

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(bfs_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='bfs', weight=4, color='blue'))
fmap

In [ ]:
def dfs(start, end):
  path = []
  dist = float(0)
  num_visited = int(0)

  previous_node = {}
  dfs_stack = []

  dfs_stack.append(start)
  
  while dfs_stack:
    num_visited += 1

    node = dfs_stack.pop()

    if (node == end):
      break

    if node not in edges_distance:
      continue

    for destination in edges_distance[node]:
      if destination == start or destination in previous_node:
        continue
      dfs_stack.append(destination)
      previous_node[destination] = node

  path.insert(0,end)
  while path[0] in previous_node:
    dist += edges_distance[previous_node[path[0]]][path[0]]
    path.insert(0, previous_node[path[0]])
  
  return path, dist, num_visited

# Show the result of DFS
dfs_path, dfs_dist, dfs_visited = dfs(start, end)
print(f'The number of nodes in the path found by DFS: {len(dfs_path)}')
print(f'Total distance of path found by DFS: {dfs_dist} m')
print(f'The number of visited nodes in DFS: {dfs_visited}\n')

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(dfs_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='dfs', weight=4, color='green'))
fmap

In [ ]:
import heapq

def ucs(start, end):
  path = []
  dist = float(0)
  num_visited = int(0)

  previous_node = {}
  heap = []

  heapq.heappush(heap, (float(0), start))
  
  while heap:
    num_visited += 1

    distance, node = heapq.heappop(heap)

    if (node == end):
      break

    if node not in edges_distance:
      continue

    for destination in edges_distance[node]:
      if destination == start or destination in previous_node and previous_node[destination][1] < distance + edges_distance[node][destination]:
        continue
      heapq.heappush(heap, (distance + edges_distance[node][destination], destination))
      previous_node[destination] = (node, distance + edges_distance[node][destination])

  path.insert(0,end)
  while path[0] in previous_node:
    dist += edges_distance[previous_node[path[0]][0]][path[0]]
    path.insert(0, previous_node[path[0]][0])
  
  return path, dist, num_visited

# Show the result of UCS
ucs_path, ucs_dist, ucs_visited = ucs(start, end)
print(f'The number of nodes in the path found by UCS: {len(ucs_path)}')
print(f'Total distance of path found by UCS: {ucs_dist} m')
print(f'The number of visited nodes in UCS: {ucs_visited}\n')

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(ucs_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='ucs', weight=4, color='violet'))
fmap

In [ ]:
import heapq

def astar(start, end):
  path = []
  dist = float(0)
  num_visited = int(0)

  previous_node = {}
  heap = []
  
  heapq.heappush(heap, (heuristic_distance[start][end], start))
  
  while heap:
    num_visited += 1

    distance, node = heapq.heappop(heap)
    distance -= heuristic_distance[node][end]

    if (node == end):
      break

    if node not in edges_distance:
      continue

    for destination in edges_distance[node]:
      if destination == start or destination in previous_node and previous_node[destination][1] < distance + edges_distance[node][destination] + heuristic_distance[destination][end]:
        continue
      heapq.heappush(heap, (distance + edges_distance[node][destination] + heuristic_distance[destination][end], destination))
      previous_node[destination] = (node, distance + edges_distance[node][destination] + heuristic_distance[destination][end])

  path.insert(0,end)
  while path[0] in previous_node:
    dist += edges_distance[previous_node[path[0]][0]][path[0]]
    path.insert(0, previous_node[path[0]][0])
  
  return path, dist, num_visited

# Show the result of A* search
astar_path, astar_dist, astar_visited = astar(start, end)
print(f'The number of nodes in the path found by A* search: {len(astar_path)}')
print(f'Total distance of path found by A* search: {astar_dist} m')
print(f'The number of visited nodes in A* search: {astar_visited}\n')

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(astar_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='astar', weight=4, color='red'))
fmap

In [ ]:
import heapq

def astar_time(start, end):
  path = []
  time = float(0)
  num_visited = int(0)

  previous_node = {}
  heap = []

  heapq.heappush(heap, (heuristic_distance[start][end] / maximum_speed_limit, start))
  
  while heap:
    num_visited += 1

    t, node = heapq.heappop(heap)
    t -= heuristic_distance[node][end] / maximum_speed_limit

    if (node == end):
      break

    if node not in edges_distance:
      continue

    for destination in edges_distance[node]:
      if destination == start or destination in previous_node and previous_node[destination][1] < (t + edges_distance[node][destination] / speed_limit[node][destination] + heuristic_distance[destination][end] / maximum_speed_limit):
        continue
      heapq.heappush(heap, ((t + edges_distance[node][destination] / speed_limit[node][destination] + heuristic_distance[destination][end] / maximum_speed_limit), destination))
      previous_node[destination] = (node, (t + edges_distance[node][destination] / speed_limit[node][destination] + heuristic_distance[destination][end] / maximum_speed_limit))

  path.insert(0,end)
  while path[0] in previous_node:
    time += edges_distance[previous_node[path[0]][0]][path[0]] / speed_limit[previous_node[path[0]][0]][path[0]]
    path.insert(0, previous_node[path[0]][0])
  
  return path, time, num_visited

# Show the shortest time result of A* search
time_path, time, time_visited = astar_time(start, end)
print(f'The number of nodes in the path found by A* search: {len(time_path)}')
print(f'Total second of path found by A* search: {time} s')
print(f'The number of visited nodes in A* search: {time_visited}\n')

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(time_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='astar', weight=4, color='red'))
fmap